In [ ]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
import threading
import time
import random


class IBApi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.price = None
        self.price_received = threading.Event()
        self.connected = threading.Event()
    
    def error(self, reqId, errorCode, errorString, advancedOrderRejectJson="", arg5=""):
        # Filter out informational messages (codes 2100-2110 are typically info, not errors)
        if errorCode >= 2100 and errorCode < 2200:
            print(f"Info {errorCode}: {errorString}")
        else:
            print(f"Error {errorCode}: {errorString}")
    
    def nextValidId(self, orderId):
        """Called when connection is established"""
        super().nextValidId(orderId)
        self.connected.set()
        print("Connection established, ready to request data")
    
    def tickPrice(self, reqId, tickType, price, attrib):
        """Receives price updates from IB"""
        if tickType == 4:  # Last price
            print(f"GOOGL Current Price: ${price:.2f}")
            self.price = price
            self.price_received.set()
        elif tickType == 1:  # Bid price (fallback if no last price)
            if not self.price_received.is_set():
                print(f"GOOGL Bid Price: ${price:.2f}")
                self.price = price
                self.price_received.set()
        elif tickType == 2:  # Ask price (fallback)
            if not self.price_received.is_set():
                print(f"GOOGL Ask Price: ${price:.2f}")
                self.price = price
                self.price_received.set()
    
    def tickSize(self, reqId, tickType, size):
        """Receives size updates from IB"""
        pass


def run_loop(app):
    """Run the IB connection in a separate thread"""
    app.run()


def twsDisconnect(app):
    """Disconnect from TWS/Gateway"""
    print("Disconnecting from Interactive Brokers...")
    app.disconnect()
    time.sleep(1)
    print("Disconnected successfully")


def main():
    # Create the IB API app
    app = IBApi()
    
    # Use a random client ID to avoid conflicts
    client_id = random.randint(1000, 9999)
    
    # Connect to IB Gateway or TWS
    # Default ports: 7497 for TWS paper trading, 7496 for TWS live, 4002 for Gateway paper, 4001 for Gateway live
    print("Connecting to Interactive Brokers...")
    try:
        app.connect("127.0.0.1", 7497, clientId=client_id)
        print(f"Connection request sent to 127.0.0.1:7497 with clientId={client_id}")
    except Exception as e:
        print(f"Connection failed with exception: {e}")
        return
    
    # Start the connection thread
    api_thread = threading.Thread(target=run_loop, args=(app,), daemon=True)
    api_thread.start()
    
    print("API thread started, checking if connected...")
    
    # Wait for connection to be fully established
    print("Waiting for connection to be established...")
    if not app.connected.wait(timeout=10):
        print("Failed to establish connection.")
        print("\nTroubleshooting:")
        print("1. Make sure TWS or IB Gateway is running and logged in")
        print("2. Verify API settings are enabled:")
        print("   - In TWS: File → Global Configuration → API → Settings")
        print("   - Check 'Enable ActiveX and Socket Clients'")
        print("   - Check 'Allow connections from localhost only'")
        print(f"3. Verify the port number (currently using 7497)")
        print("   - TWS Paper: 7497")
        print("   - TWS Live: 7496")
        print("   - Gateway Paper: 4002")
        print("   - Gateway Live: 4001")
        print(f"4. Check if clientId=1 is already in use")
        print(f"5. Check if you see any error messages above")
        app.disconnect()
        return
    
    time.sleep(2)  # Additional wait for data connections to stabilize
    
    # Create a contract for GOOGL stock
    contract = Contract()
    contract.symbol = "GOOGL"
    contract.secType = "STK"
    contract.exchange = "SMART"
    contract.currency = "USD"
    contract.primaryExchange = "NASDAQ"
    
    # Request market data
    print("Requesting market data for GOOGL...")
    app.reqMktData(1, contract, "", False, False, [])
    
    # Wait for price data (timeout after 15 seconds)
    if app.price_received.wait(timeout=15):
        print(f"\nSuccessfully retrieved GOOGL price: ${app.price:.2f}")
    else:
        print("\nTimeout waiting for price data.")
        print("Possible issues:")
        print("- Market data subscription not active for NASDAQ stocks")
        print("- Markets are closed (try during market hours)")
        print("- Contract definition issue")
        print("\nTry requesting delayed data by enabling it in TWS: Edit → Global Configuration → Market Data → Use delayed market data")
    
    # Cancel market data and disconnect
    app.cancelMktData(1)
    time.sleep(1)
    twsDisconnect(app)

if __name__ == "__main__":
    main()
    